참고 링크

https://lsjsj92.tistory.com/570?category=853217

https://eda-ai-lab.tistory.com/528?category=736098


In [2]:
#import
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [5]:
##구글드라이브 권한 획득
!apt install -y -qq --fix-broken python-pycurl python-apt 2> /dev/null
!apt install -y -qq software-properties-common module-init-tools 2> /dev/null
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt update -qq 2>&1 > /dev/null
!apt install -y -qq google-drive-ocamlfuse fuse 2> /dev/null

from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
creds = GoogleCredentials.get_application_default()

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

from getpass import getpass

vcode = getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}



python-pycurl is already the newest version (7.43.0.1-0.2).
python-apt is already the newest version (1.6.5ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.
software-properties-common is already the newest version (0.96.24.32.14).
module-init-tools is already the newest version (24-1ubuntu3.5).
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.


fuse is already the newest version (2.9.7-1ubuntu1).
google-drive-ocamlfuse is already the newest version (0.7.24-0ubuntu1~ubuntu18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?cli

In [6]:
#drive에 경로 생성
!mkdir -p google_drive
!google-drive-ocamlfuse -o nonempty google_drive

In [7]:
userData = pd.read_csv("google_drive/steamData/userOwnedResult.csv")
userData

,Unnamed: 0,appid,playtime_forever,playtime_2weeks,steamid,weight,newsteamid
0,0,10,-inf,NaN,76561197960265744,1,0
1,1,919670,-inf,NaN,76561197960265744,1,0
2,2,257510,0.667453,NaN,76561197960265744,2,0
3,3,319630,-inf,NaN,76561197960265744,1,0
4,4,45740,-inf,NaN,76561197960265744,1,0
...,...,...,...,...,...,...,...
1913313,1913313,214950,-inf,NaN,76561197964555948,1,8397
1913314,1913314,254320,-inf,NaN,76561197964555948,1,8397
1913315,1913315,243450,-inf,NaN,76561197964555948,1,8397
1913316,1913316,255370,-1.301030,NaN,76561197964555948,-1,8397


In [8]:
#outlier제거
userData = userData.drop(columns=['Unnamed: 0'])
userData = userData.drop_duplicates(['appid', 'steamid'])

In [9]:
userData.head()

,appid,playtime_forever,playtime_2weeks,steamid,weight,newsteamid
0,10,-inf,NaN,76561197960265744,1,0
1,919670,-inf,NaN,76561197960265744,1,0
2,257510,0.667453,NaN,76561197960265744,2,0
3,319630,-inf,NaN,76561197960265744,1,0
4,45740,-inf,NaN,76561197960265744,1,0


In [10]:
#유저수 확인
userData['newsteamid'].unique().size

8398

In [11]:
#검색할 유저 정보 추가
searchData = pd.DataFrame(columns=['appid',	'playtime_forever',	'playtime_2weeks',	'steamid'])
userid = 76561198406547455

url = 'https://api.steampowered.com/IPlayerService/GetOwnedGames/v1/?key=F1DAAB7BF3894FBDC83881C81C20803A&steamid=' + str(userid)
response = requests.get(url)

flag = 0

if response.status_code == 200:
    flag = 1
    html = response.text
    soup = BeautifulSoup(html, 'html.parser') 
    jsonData = pd.read_json(soup.text)
    if jsonData.empty:
      print("empty")
    elif jsonData['response']['game_count'] == 0:
      print("none data")
else :
  print(response.status_code)
if flag == 1:
  searchUser = pd.DataFrame(jsonData['response']['games'])
  searchUser = searchUser.drop(['playtime_windows_forever', 'playtime_mac_forever', 'playtime_linux_forever'], axis='columns')
  searchUser['steamid'] = userid
  searchUser['newsteamid'] = 8398
else:
  searchUser = pd.DataFrame()
searchUser

,appid,playtime_forever,playtime_2weeks,steamid,newsteamid
0,361280,0,NaN,76561198406547455,8398
1,413150,7175,NaN,76561198406547455,8398
2,437920,646,34.0,76561198406547455,8398
3,582500,274,NaN,76561198406547455,8398
4,677160,404,NaN,76561198406547455,8398


In [12]:
#weight 할당 def
def nonePlaying(playtime_forever, playtime_2weeks, weight):
  #플레이시간 0
  if playtime_forever == float('-inf'):
    weight = 1
  #플레이시간 1시간 이내
  elif playtime_forever < 0:
    weight = -1
  #플레이시간 10시간 이내
  # elif 0 <= playtime_forever < 1:
  #   weight = 2
  #플레이시간 1시간 이상
  else :
    weight = 2

  if playtime_2weeks >= 60:
    weight = weight + 1
  return weight


In [13]:
#가중치 부여
searchUser['weight'] = 0
searchUser['playtime_forever'] = searchUser['playtime_forever']/60
searchUser['playtime_forever'] = np.log10(searchUser['playtime_forever'])
searchUser['weight'] = searchUser.apply(lambda x : nonePlaying(x['playtime_forever'],x['playtime_2weeks'], x['weight']), axis = 1)
searchUser

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,appid,playtime_forever,playtime_2weeks,steamid,newsteamid,weight
0,361280,-inf,NaN,76561198406547455,8398,1
1,413150,2.077671,NaN,76561198406547455,8398,2
2,437920,1.032081,34.0,76561198406547455,8398,2
3,582500,0.659599,NaN,76561198406547455,8398,2
4,677160,0.828230,NaN,76561198406547455,8398,2


In [14]:
#데이터 병합
userData = userData.append(searchUser)
userData

,appid,playtime_forever,playtime_2weeks,steamid,weight,newsteamid
0,10,-inf,NaN,76561197960265744,1,0
1,919670,-inf,NaN,76561197960265744,1,0
2,257510,0.667453,NaN,76561197960265744,2,0
3,319630,-inf,NaN,76561197960265744,1,0
4,45740,-inf,NaN,76561197960265744,1,0
...,...,...,...,...,...,...
0,361280,-inf,NaN,76561198406547455,1,8398
1,413150,2.077671,NaN,76561198406547455,2,8398
2,437920,1.032081,34.0,76561198406547455,2,8398
3,582500,0.659599,NaN,76561198406547455,2,8398


In [15]:
#유저수 확인
userData['newsteamid'].unique().size

8399

In [16]:
#applist 불러오기
appList = pd.read_csv("google_drive/steamData/applist.csv")
appList

,appid,name
0,216938,Pieterw test app76 ( 216938 )
1,660010,test2
2,660130,test3
3,1118314,NaN
4,485070,Dyna Bomb Demo
...,...,...
111304,1052230,CONCLUSE 2
111305,1371580,Myth of Empires
111306,1163140,Shanghai Office Simulator
111307,1101910,Warriors: Rise to Glory! Online Multiplayer Op...


In [17]:
#app검색
appList[appList['appid'] == 730]

,appid,name
84756,730,Counter-Strike: Global Offensive


In [18]:
#pivot table 생성
pivotUserApp = userData.pivot(
    index = 'newsteamid',
    columns = 'appid',
    values = 'weight'
).fillna(0)

In [19]:
pivotUserApp

appid,10,20,30,40,50,60,70,80,100,130,220,240,280,300,320,340,360,380,400,420,500,550,620,730,1002,1200,1210,1230,1250,1280,1300,1309,1313,1320,1500,1510,1520,1525,1530,1600,...,1476340,1476520,1477480,1477490,1477830,1478650,1480340,1480400,1483780,1486350,1487780,1489830,1489840,1490410,1493020,1494440,1494730,1495320,1497820,1502380,1503530,1504150,1505620,1507210,1507720,1508740,1508750,1513420,1518970,1525620,1530870,1537670,1537710,1540150,1542950,1546500,1546540,1557060,1558430,1558440
newsteamid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,2.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,1.0,0.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,1.0,1.0,1.0,1.0,-1.0,2.0,2.0,1.0,1.0,2.0,2.0,-1.0,2.0,2.0,1.0,1.0,-1.0,1.0,-1.0,0.0,2.0,2.0,2.0,0.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,-1.0,2.0,-1.0,2.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8394,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8395,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8396,2.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
#인기도 비례 점수 재책정
matrix = pivotUserApp.values

weightMean = np.mean(matrix, axis = 1)

matrixUserMean = matrix - weightMean.reshape(-1, 1)

In [21]:
matrix

array([[ 1.,  1.,  1., ...,  0.,  0.,  0.],
       [-1.,  1.,  1., ...,  0.,  0.,  0.],
       [ 2.,  1.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 2.,  1.,  1., ...,  0.,  0.,  0.],
       [ 2., -1.,  2., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [22]:
pd.DataFrame(matrixUserMean, columns = pivotUserApp.columns)

appid,10,20,30,40,50,60,70,80,100,130,220,240,280,300,320,340,360,380,400,420,500,550,620,730,1002,1200,1210,1230,1250,1280,1300,1309,1313,1320,1500,1510,1520,1525,1530,1600,...,1476340,1476520,1477480,1477490,1477830,1478650,1480340,1480400,1483780,1486350,1487780,1489830,1489840,1490410,1493020,1494440,1494730,1495320,1497820,1502380,1503530,1504150,1505620,1507210,1507720,1508740,1508750,1513420,1518970,1525620,1530870,1537670,1537710,1540150,1542950,1546500,1546540,1557060,1558430,1558440
0,0.991551,0.991551,0.991551,0.991551,0.991551,0.991551,1.991551,0.991551,0.991551,0.991551,1.991551,0.991551,0.991551,0.991551,0.991551,-1.008449,0.991551,0.991551,-1.008449,0.991551,0.991551,-1.008449,1.991551,-1.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,...,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449,-0.008449
1,-1.068711,0.931289,0.931289,0.931289,0.931289,0.931289,0.931289,0.931289,0.931289,0.931289,-1.068711,0.931289,0.931289,0.931289,-1.068711,0.931289,0.931289,0.931289,-1.068711,0.931289,0.931289,0.931289,0.931289,1.931289,-0.068711,0.931289,-0.068711,0.931289,0.931289,0.931289,-0.068711,-0.068711,-0.068711,-0.068711,-1.068711,-1.068711,0.931289,-0.068711,-1.068711,-0.068711,...,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711,-0.068711
2,1.937735,0.937735,0.937735,0.937735,0.937735,-1.062265,1.937735,1.937735,0.937735,0.937735,1.937735,1.937735,-1.062265,1.937735,1.937735,0.937735,0.937735,-1.062265,0.937735,-1.062265,-0.062265,1.937735,1.937735,1.937735,-0.062265,0.937735,-0.062265,0.937735,1.937735,0.937735,-0.062265,-0.062265,-0.062265,-0.062265,0.937735,0.937735,0.937735,-0.062265,0.937735,-0.062265,...,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265,-0.062265
3,1.958197,1.958197,1.958197,0.958197,1.958197,0.958197,1.958197,1.958197,1.958197,1.958197,1.958197,2.958197,1.958197,1.958197,1.958197,0.958197,0.958197,-1.041803,1.958197,-1.041803,1.958197,1.958197,1.958197,2.958197,-0.041803,-0.041803,-0.041803,-0.041803,1.958197,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,...,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803,-0.041803
4,1.985064,0.985064,0.985064,0.985064,0.985064,0.985064,0.985064,1.985064,0.985064,0.985064,-0.014936,1.985064,-0.014936,-0.014936,0.985064,0.985064,0.985064,0.985064,0.985064,0.985064,1.985064,1.985064,0.985064,1.985064,-0.014936,-0.014936,-0.014936,-0.014936,1.985064,-0.014936,-0.014936,-0.014936,-0.014936,-0.014936,-0.014936,-0.014936,-0.014936,-0.014936,-0.014936,-0.

In [23]:
#scipy에서 제공해주는 svd이용
U, sigma, Vt = svds(matrixUserMean, k = 12)

In [24]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(8399, 12)
(12,)
(12, 24974)


In [25]:
#대칭행렬로 변환
sigma = np.diag(sigma)

In [26]:
sigma.shape

(12, 12)

In [27]:
#U, sigma, Vt의 내적 수행하여 원본행렬로 복원
svd_user_predicted_weight = np.dot(np.dot(U, sigma), Vt) + weightMean.reshape(-1, 1)

In [28]:
svd_preds = pd.DataFrame(svd_user_predicted_weight, columns=pivotUserApp.columns)
svd_preds

appid,10,20,30,40,50,60,70,80,100,130,220,240,280,300,320,340,360,380,400,420,500,550,620,730,1002,1200,1210,1230,1250,1280,1300,1309,1313,1320,1500,1510,1520,1525,1530,1600,...,1476340,1476520,1477480,1477490,1477830,1478650,1480340,1480400,1483780,1486350,1487780,1489830,1489840,1490410,1493020,1494440,1494730,1495320,1497820,1502380,1503530,1504150,1505620,1507210,1507720,1508740,1508750,1513420,1518970,1525620,1530870,1537670,1537710,1540150,1542950,1546500,1546540,1557060,1558430,1558440
0,1.077199,0.662609,0.623848,0.688276,0.731539,0.668627,0.612992,0.271185,0.260921,0.711311,0.624451,0.884256,0.156430,0.344464,0.447565,0.382150,0.306122,0.326379,0.405662,0.337919,0.532913,0.776042,0.618390,1.540657,0.006372,0.035370,0.000098,0.047957,0.257420,0.044038,0.056824,0.066699,0.063314,0.002591,-0.017387,-0.012939,-0.026787,-0.001066,-0.017387,0.003801,...,-0.000070,-0.000070,-0.000070,-0.000070,-0.000535,-0.000427,-0.000513,-0.000301,0.000421,-0.001451,0.002098,-0.001429,-0.001429,-0.000864,-0.000977,-0.000592,-0.001200,-0.001714,-0.000870,-0.000070,-0.001442,-0.000655,-0.000393,-0.000886,-0.000070,-0.000110,-0.000110,-0.000070,0.002098,-0.000702,0.005520,-0.000694,0.019440,-0.001257,0.002514,-0.001830,-0.000524,-0.001000,-0.000028,-0.000028
1,0.820881,0.711193,0.699710,0.801512,0.751715,0.759883,0.571853,0.282835,0.293702,0.789821,0.706239,0.688955,0.156283,0.523845,0.521969,0.507917,0.474947,0.390551,0.407929,0.430971,0.845182,0.989097,0.853154,1.328564,0.074697,0.285873,0.033618,0.294999,0.752643,0.316046,0.216812,0.236003,0.197553,0.034896,0.262421,0.266628,0.125063,0.031690,0.249988,0.118192,...,0.035322,0.035322,0.035322,0.035322,0.032321,0.031121,0.033984,0.032892,0.034128,0.037545,0.027822,0.038807,0.038807,0.032778,0.040588,0.044545,0.038810,0.031175,0.030750,0.035322,0.033053,0.041655,0.032509,0.034674,0.035322,0.032780,0.032780,0.035322,0.027822,0.032335,0.033269,0.034077,0.057709,0.031924,0.033926,0.030843,0.038911,0.039426,0.036069,0.036069
2,1.767328,0.824352,0.879099,0.845203,1.004775,0.647643,0.826030,0.560177,0.582318,1.109432,1.160273,1.519111,0.361527,1.321850,0.710888,0.616861,0.689763,0.553201,0.781421,0.618329,1.358315,1.983245,1.522633,2.331130,0.091540,0.655310,-0.000801,0.672947,1.817023,0.794169,0.315160,0.372218,0.307562,0.003336,0.703240,0.713644,0.500125,-0.002404,0.683371,0.207245,...,0.007840,0.007840,0.007840,0.007840,-0.002832,-0.003709,0.005419,0.000946,-0.009913,0.011048,-0.010906,0.002036,0.002036,-0.001362,-0.000309,0.000283,0.000653,-0.001862,-0.000234,0.007840,-0.006436,0.006993,-0.002853,-0.004988,0.007840,-0.002241,-0.002241,0.007840,-0.010906,-0.003278,-0.014423,-0.003837,0.017314,-0.003945,-0.021909,-0.004175,0.007263,0.002307,0.005442,0.005442
3,2.015936,1.119012,1.185743,1.180684,1.435652,1.006935,1.053524,1.305508,1.268956,1.406281,2.538313,3.253608,1.014382,1.917745,1.665374,1.270647,1.607664,1.879134,2.323294,1.925079,2.329889,3.257379,2.893656,3.155298,0.082126,0.379752,0.009477,0.418263,1.590636,0.489977,0.252449,0.302841,0.215308,0.014358,0.376321,0.348341,0.295196,0.008164,0.308613,0.076080,...,0.008417,0.008417,0.008417,0.008417,0.007751,0.007778,0.011861,0.007991,-0.001602,0.002795,-0.003243,0.003132,0.003132,0.005666,-0.003794,0.003930,0.001657,0.005474,0.005973,0.008417,0.003390,0.004755,0.006594,0.004431,0.008417,0.006800,0.006800,0.008417,-0.003243,0.003666,-0.007489,0.002652,0.029128,0.004549,-0.005983,0.003993,0.003010,0.001074,0.004452,0.004452
4,1.642555,0.898405,0.858293,0.946006,1.093626,0.871579,0.817229,0.754181,0.726014,1.053338,1.530815,2.152587,0.517769,1.066434,1.038031,0.802003,0.889199,1.023123,1.244477,1.069321,1.517905,2.085505,1.743409,2.602165,0.040031,0.177100,0.002507,0.217029,0.857809,0.241750,0.171074,0.208712,0.162576,0.006946,0.098589,0.093132,0.065814,0.000004,0.067057,0.026841,...,0.001698,0.001698,0.001698,0.001698,0.000866,0.001754,0.002951,0.000726,-0.002568,-0.005387,-0.001828,-0.004520,-0.004520,-0.000384,-0.006839,-0.003873,-0.004617,-0.

In [29]:
#유저id, applist, weight 테이블을 입력
#사용자 아이디에 SVD로 나온 결과의 weight가 높은 순으로 데이터 정렬
#사용자가 소유한 게임 제외
#사용자가 소유하지 않은 게임중 weight가 높은 것을 추천

def recommend_games(svd_preds, user_id, ori_applist, ori_users, num_recommendations=5):
    
    user_row_number = user_id
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 게임 데이터 정렬 -> 게임 weight가 높은 순으로 정렬 됌
    sorted_user_predictions = svd_preds.iloc[user_row_number].sort_values(ascending=False)
    
    # 원본 데이터에서 user_id 해당하는 데이터를 뽑아낸다. 
    user_data = ori_users[ori_users.newsteamid == user_id]
    
    # 위에서 뽑은 user_data와 게임 데이터를 합친다. 
    user_history = user_data.merge(ori_applist, on = 'appid').sort_values(['weight'], ascending=False)
    
    # 원본 게임 데이터에서 사용자 목록에 있는 게임 데이터를 제외해 데이터를 추출
    recommendations = ori_applist[~ori_applist['appid'].isin(user_history['appid'])]
    # 사용자의 게임 weight가 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'appid')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [30]:
already_rated, predictions = recommend_games(svd_preds, 8398, appList, userData, 30)

In [31]:
already_rated

,appid,playtime_forever,playtime_2weeks,steamid,weight,newsteamid,name
1,413150,2.077671,NaN,76561198406547455,2,8398,Stardew Valley
2,437920,1.032081,34.0,76561198406547455,2,8398,Tricky Towers
3,582500,0.659599,NaN,76561198406547455,2,8398,We Were Here
4,677160,0.828230,NaN,76561198406547455,2,8398,We Were Here Too
0,361280,-inf,NaN,76561198406547455,1,8398,Turmoil


In [32]:
predictions

,appid,name,Predictions
23543,892970,Valheim,0.053929
22649,252950,Rocket League,0.050328
13914,238960,Path of Exile,0.045393
14695,105600,Terraria,0.044939
22956,646570,Slay the Spire,0.039703
23129,945360,Among Us,0.039178
14408,211820,Starbound,0.038696
24184,322330,Don't Starve Together,0.038001
21473,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,0.037268
14372,212680,FTL: Faster Than Light,0.036773


In [33]:
pivotUserApp.iloc[4]

appid
10         2.0
20         1.0
30         1.0
40         1.0
50         1.0
          ... 
1546500    0.0
1546540    0.0
1557060    0.0
1558430    0.0
1558440    0.0
Name: 4, Length: 24974, dtype: float64